In [11]:
import requests
import zipfile
import io

# URL of the zip file
url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"

# Download the file
response = requests.get(url)
response.raise_for_status()  # Check if the download was successful

# Extract the zip file
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall("data")

print("Download and extraction completed.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device
     

Download and extraction completed.


'cpu'

In [12]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms, datasets

# Create a transforms pipeline
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])



In [13]:
from pathlib import Path
from torch.utils.data import DataLoader
import os


train_dir = Path("data/train")
test_dir = Path("data/test")

train_data = datasets.ImageFolder(root=train_dir, transform=simple_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=simple_transform)

NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS)
class_names = train_data.classes
print(class_names)


['pizza', 'steak', 'sushi']


In [14]:
model_0 = torchvision.models.efficientnet_b0(pretrained=True).to(device)

In [15]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model_0.features.parameters():
    param.requires_grad = False

In [16]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model_0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(device)